In [ ]:
from statsbomb_data_extraction import Game, fetch_matches_for_season, fetch_all_seasons_for_league
from draw_pitch import draw_pitch

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
from scipy.stats import uniform

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures,OneHotEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, log_loss
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

# xG Model

The concept of "Expected Goals" (xG for short), is a widely used metric among soccer analysts and fans alike. In fact, the idea of xG has infiltrated the mainstream in recent years, to the point where now, even most casual fans have probably heard of the term. The xG metric essentially describes how many goals we'd expect a team to score based on the chances it creates. Each shot a team takes can be assigned an xG value, ranging from 0 to 1, which represents the probability of that shot resulting in a goal. 

The xG metric helps provide objectivity in the analysis of a soccer match, since actual score lines can often be misleading given that luck plays such a large role in the game. If we can see that a team dominated a game by creating many more chances and generating a much larger xG than the other team, this tells us they were probably better on that day, regardless of what the final scoreline may say. Additionally, analyzing xG can help inform play styles, in terms of determining what type of shots are better to take to maximize the probability of scoring a goal.  

This notebook consists in a comprehensive overview of an xG model created using Statsbomb public data. More specifically, the model is creating using data from Barcelona's La Liga matches from 2004 to 2016.

## Notebook Overview

<font size = 4>

1. Fetching data 
    - 1.1 Getting data from Statsbomb
    - 1.2 Extracting shots data and attributes
    
    
2. Data exploration
    - 2.1 Creating new variables
    - 2.2 Separate test and training set
    - 2.3 Visualizing relationships

    
3. Data preprocessing for ML models
    - 2.1 Categorical variables
    - 2.2 Numerical variables
    - 2.3 Combined pipeline
    
    
4. Models
    - 4.1 Logistic regression

## 1. Fetching data

### 1.1 Getting data from Statsbomb. 

To start off, I wrote some functions in the `statsbomb_data_extraction.py` file to get data from statsbomb in the form of json files for each game in a desired season / league. In the cell below, I call `fetch_matches_for_league` with argument **11**, as this corresponds to the La Liga comp in Statsbomb public data. (See [here](https://github.com/statsbomb/open-data/blob/master/data/competitions.json)).

In [ ]:
la_liga = fetch_all_seasons_for_league(11)

So we have a data structure, called `la_liga`, which has league season names as the keys of the outer dictionary. The values are dictionaries, which map game ids (as specified by statsbomb) to `Game` objects (see `statsbomb_data_extraction.py` file for Game object definition).

### 1.2 Getting shots data 

Now I will write some functions to parse through the json of each game, and extract all the shots, as well as features related to the shots. In particular, these features are: 

- **play pattern**: pattern of play which led to the shot
- **x start location**: x-location of the shot 
- **y start location**: y-location of the shot 
- **duration**: duration of the shot
- **outcome**: result of the shot
- **technique**: technique with which the shot was hit
- **first time**: whether the shot was hit for time or not
- **x gk position**: x-location of the gk when shot was taken
- **y gk position**: y-location of the gk when shot was taken
- **type of shot**: whether shot was from open play or set piece (and type of set piece specified)
- **num opponents within 5 yards**: number of opponents which were within 5 yards of shot location
- **num opponents between shot and goal**: number of opponents which were between the shot location, and the lines connecting shot location and the two posts

I will also extract some information related to the shot that will be useful to reference the shots, but will not be used to train ML model. These include:
- **game id in which shot was taken**: pattern of play which led to the shot
- **Season of game in which shot was taken**: x-location of the shot 
- **statsbomb xg**: xg given by statsbomb to compare to my values later on

All the variables listed above can be extracted by parsing through the json directly, except for the last one. Therefore, below, I will write a function to get this variable.

In [ ]:
def check_player_btwn_shot_and_goal(x_shot, y_shot, x_player, y_player):
    """
    Function which checks whether a player in a stats bomb freeze frame is between the shot location, and the two lines
    connecting the shot location to the posts. See here for coordinate specifications: 
    https://github.com/statsbomb/open-data/blob/master/doc/StatsBomb%20Open%20Data%20Specification%20v1.1.pdf
    
    Arguments:
    x_shot    x-location of shot
    y_shot    y-location of shot
    x_player  x-location of player
    y_player  y-location of player    
    """
    x_diff = x_player - x_shot
    y_diff = y_player - y_shot
    
    if 120 - x_shot == 0:
        return False
    
    slope_1 = (36 - y_shot) / (120 - x_shot)
    slope_2 = (44 - y_shot) / (120 - x_shot) 
                    
    return (x_diff >= 0) and ((y_shot + slope_1 * x_diff) < y_player < (y_shot + slope_2 * x_diff))


def plot_shot_freeze_frame(game_json, shot_id, ax):
    player_pos_list_x = []
    player_pos_list_y = []
    x_shot = 0
    y_shot = 0
    
    for events in game_json:
        if events['id'] == shot_id:
            x_shot = events['location'][0]
            y_shot = events['location'][1]
        
            for players in events['shot']['freeze_frame']:
                if (players['teammate'] == False):
                    player_pos_list_x.append(players['location'][0])
                    player_pos_list_y.append(players['location'][1])
            
    ax.scatter(player_pos_list_x, player_pos_list_y)
    ax.scatter(x_shot, y_shot, s=100)
    ax.plot([x_shot, 120], [y_shot,36], color = 'red', linestyle = '--')
    ax.plot([x_shot, 120], [y_shot,44], color = 'red', linestyle = '--')

Lets call the `plot_shot_freeze_frame` function to visualize the _num opponents between shot and goal_ feature. The shot below would have a value of 2 for this feature.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12,8))
draw_pitch(axis=ax, rotate=True)
plot_shot_freeze_frame(la_liga["2018/2019"]['16215'].json_file, "6c30cac4-6f3b-4cd0-9913-26137b058416", ax=ax)

Now I will write a function which takes a json file for a given game, and returns a data frame containing all shots taken along with the variables related to that shot. Then I will write two wrapper function: `get_shots_for_season` which calls `get_shots_for_game` on all game files for that season, and `get_shots_for_league`, which does a similar thing, but for each game across every season we have for that league.

In [ ]:
def get_shots_for_game(game_json):
    """
    Function which parses through a game JSON and return a data frame containing all shots taken in that game with several features 
    related to that shot
    
    Arguments
    game_json - event level json for a game
    """
    
    #features for each shot, which will be the columns of our data frame
    shot_id_list = []
    x_start_location_list = []
    y_start_location_list = []
    play_pattern_list = []
    duration_list = []
    outcome_list = []
    technique_list = []
    type_shot_list = []
    first_time_list = []
    x_gk_pos_list = []
    y_gk_pos_list = []
    num_opponents_5_yards_list = []
    num_opponents_between_goal_list = []
    
    statsbomb_xg = [] #to compare to my model
    
    #-------------------------#

    for events in game_json:
    
        if events['type']['name'] == 'Shot':
        
            #get data for first 8 features
            shot_id = events['id']
            x_start_location = events['location'][0]
            y_start_location = events['location'][1]
            play_pattern = events['play_pattern']['name']
            duration = events['duration']
            outcome = events['shot']['outcome']['name'] 
            technique = events['shot']['technique']['name']
            type_shot = events['shot']['type']['name']
            xg = events['shot']['statsbomb_xg']
        
            #check if json shot has a first_time attribute, if not set first_time to False
            if 'first_time' in events['shot']:
                first_time = events['shot']['first_time']
            else:
                first_time = False
            
            #check if shot has a freeze_frame dictionary
            if "freeze_frame" in events["shot"]:
                
                num_opponents_5_yards = 0
                num_opponents_between_goal = 0
                
                for player in events["shot"]["freeze_frame"]:
                    x_player = player['location'][0]
                    y_player = player['location'][1]
                    
                    #count how many opponents were within 5 yards of player when shot was taken
                    if ((x_start_location - x_player)**2 + (y_start_location - y_player)**2) <= 25 and (player['teammate'] == False):
                        if (player['position']['name'] != 'Goalkeeper'):
                            num_opponents_5_yards += 1
                    
                    #count how many opponents were between shot and goal
                    if (player['teammate'] == False) and (player['position']['name'] != 'Goalkeeper'):
                        if check_player_btwn_shot_and_goal(x_start_location, y_start_location, x_player, y_player):
                            num_opponents_between_goal += 1
                    
                    #get position of opponent's goalkeeper 
                    if ((player['position']['name'] == 'Goalkeeper') and (player['teammate'] == False)):
                        x_gk_pos = player['location'][0]
                        y_gk_pos = player['location'][1]
            
            #if there is no freeze frame, assume goalkeeper is at center of goal, and 0 opponenets around shot location
            else:
                num_opponents_between_goal = 0
                num_opponents_5_yards = 0
                x_gk_pos = 120
                y_gk_pos = 40            
            
            #append data on shot to relevant list (column)
            shot_id_list.append(shot_id)
            play_pattern_list.append(play_pattern)
            x_start_location_list.append(x_start_location)
            y_start_location_list.append(y_start_location)
            duration_list.append(duration)
            outcome_list.append(outcome)
            technique_list.append(technique)
            first_time_list.append(first_time)
            x_gk_pos_list.append(x_gk_pos)
            y_gk_pos_list.append(y_gk_pos)
            type_shot_list.append(type_shot)
            num_opponents_5_yards_list.append(num_opponents_5_yards)
            num_opponents_between_goal_list.append(num_opponents_between_goal)
            statsbomb_xg.append(xg)
        
    #create data frame with column features
    shot_df = pd.DataFrame({
                       "shot id" : shot_id_list,
                       "play pattern" : play_pattern_list, 
                       "x start location" : x_start_location_list, 
                       "y start location" : y_start_location_list,
                       "duration" : duration_list, 
                       "outcome" : outcome_list, 
                       "technique" : technique_list, 
                       "first time" : first_time_list,
                       "x gk position" : x_gk_pos_list,
                       "y gk position" : y_gk_pos_list,
                       "type of shot" : type_shot_list,
                       "num opponents within 5 yards" : num_opponents_5_yards_list,
                       "num opponents between shot and goal" : num_opponents_between_goal_list,
                       "statsbomb xg" : statsbomb_xg
                       })
    
    return shot_df.set_index("shot id")

**Wrapper functions:**

In [ ]:
def get_shots_for_season(season_dict):
    """
    Takes a dictionary whichs maps game ids to Game objects, and calls get_shots_for_game() on each one.
    
    Arguments:
    season_dict - dictionary which maps game ids (string, as specified by statsbomb) to Game objects (defined above)
    """
    total_shot_df = pd.DataFrame()
    
    for keys, values in season_dict.items():
        shot_df = get_shots_for_game(values.json_file)
        shot_df["game_id"] = keys
        total_shot_df = total_shot_df.append(shot_df)
        
    return total_shot_df


def get_shots_for_league(league_dict):
    """
    Takes a dictionary which maps season names to season dictionaries for a league, and calls get_shots_for_game() on each game.
    
    Arguments:
    league_dict - dictionary which maps season names to another dictionary. This inner dictionary maps 
    game ids (as specified by statsbomb) to Game objects (defined above)
    """
    total_shot_df = pd.DataFrame()
    
    for keys, values in league_dict.items():
        shot_df = get_shots_for_season(values)
        shot_df["season_id"] = keys
        total_shot_df = total_shot_df.append(shot_df)
        print("Getting shots for " + keys)
    
    print("Done.")
    
    return total_shot_df

In [ ]:
all_la_liga_shots = get_shots_for_league(la_liga)

So we have our data frame with all shots taken in Barca games from 2004/05 to 2015/16.

In [ ]:
all_la_liga_shots

## 2. Data Exploration

Before starting to train our models, we'll want to do some data exploration to get an initial idea of what variables affect the probability of a shot resulting in a goal. Even though, as we'll see, many of the models will select the features for us, this is still a useful exercise as it will give us a more intuitive idea of what the models will be doing. 

Additionally, if we find patterns in the visualizations below that make sense based on what we know about soccer, it will be a sort of "sanity-check" that we have accurately extracted the shot data.

### 2.1 Feature engineering

We will first create the following new variables:

- Distance of shot location from goal
- Convert outcome to a binary variable (Goal = 1, Other = 0). I'm doing this since for now, I don't care if the shot was saved, went wide, etc. I only care if it went in or not.

In [ ]:
all_la_liga_shots["Distance from goal"] = np.sqrt((all_la_liga_shots["x start location"] - 120)**2 +    \
                                                  (all_la_liga_shots["y start location"] - 40)**2)

goal_label_mask = all_la_liga_shots["outcome"].str.contains("Goal")
all_la_liga_shots["outcome"].loc[goal_label_mask] = 1
all_la_liga_shots["outcome"].loc[~goal_label_mask] = 0

In [ ]:
all_la_liga_shots

### 2.2 Create a training and test set

Since I don't want the test set to influence my model in any way (including the data exploration - which could eventually inform how I choose my model), I'll split the data now into test, validation and training sets and stash the test set away until I decide on a final model.

In [ ]:
features=all_la_liga_shots.drop(["outcome"], axis=1)
targets=all_la_liga_shots["outcome"]

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(features.loc[X_train.index], targets[X_train.index], test_size=0.2)

shot_training_set = pd.merge(X_train, y_train, left_index=True, right_index=True)
shot_valid_set = pd.merge(X_valid, y_valid, left_index=True, right_index=True)
shot_test_set = pd.merge(X_test, y_test, left_index=True, right_index=True)

In [ ]:
shot_training_set

### 2.3 Visualizing relationships

Below, I write a function, `plot_percent_goal_vs_attr` to plot the percentage of goals scored by desired **categorical** variable.

In [ ]:
def plot_percent_goal_vs_attr(attribute, type_chart = "line"):
    """
    Function which plots the percentage of goals scored by desired categorical attribute in our shot training set.
    
    Arguments:
    ----------
    attribute (string) 
        - name of categorical attribute in shot training set data frame
    """
    shot_training_set_copy = shot_training_set.copy() 
    
    attr_x = attribute
    attr_y = "outcome"
    attr_agg = "shot id"
    attrs = [attr_x, attr_y, attr_agg]

    outcomes_by_attribute = shot_training_set_copy.reset_index()[attrs]   \
                            .groupby([attr_x, attr_y])               \
                            .count()                                 \
                            .unstack()                               \
                            .fillna(0)
    
    outcomes_by_attribute = outcomes_by_attribute[attr_agg].reset_index()
    outcomes_by_attribute["Percent Goal"] = outcomes_by_attribute[1] * 100  / outcomes_by_attribute.sum(axis = 1)
    outcomes_by_attribute = outcomes_by_attribute.rename(columns={0: 'No Goal', 1: 'Goal'})
    
    if type_chart == "line":
        chart = alt.Chart(outcomes_by_attribute).mark_line()
    elif type_chart == "bar":
        chart = alt.Chart(outcomes_by_attribute).mark_bar()
    
    chart = chart.encode(
        alt.X('{}:N'.format(attr_x), axis = alt.Axis(title = attr_x, labelAngle = 0)),
        alt.Y('Percent Goal:Q', axis = alt.Axis(title = 'Percent Goal'))
    ).properties(width = 850, height = 300, 
             title = "Percent Goal vs {}".format(attr_x)
    ).configure_axis(labelFontSize = 13, titleFontSize = 15
    ).configure_title(fontSize = 16)
    
    return chart

#### 2.3.1 Percent goals by num opponents between shot and goal

Now we can use our function to plot the desired variables:

In [ ]:
plot_percent_goal_vs_attr("num opponents between shot and goal", "bar")

**Comments:**

As expected, in general, the rate of goals scored decreases as more opponents are between the shot location and the goal. Interestingly, there is a bizarre peak at 6 opponents between shot and goal.

#### 2.3.2 Percent goals by num opponents within 5 yards

In [ ]:
plot_percent_goal_vs_attr("num opponents within 5 yards", "bar")

**Comments:**

We see a similar trend for rate of goals scored vs number of opponents within 5 yards of shot location.

#### 2.3.3 Percent goals by player pattern

In [ ]:
plot_percent_goal_vs_attr("play pattern", "bar")

**Comments:**

For goal scoring rate by _play pattern_, we see that "other" has a very large percent goals scored value. This tells us that "other" refers to penalties. Aside from "other", we can see that shots from counter attacks have a larger chance to produce a goal than any other play pattern, probably because counter attacks mean that the opposition is much less organized, resulting in more space. To be sure, we can check this below:

In [ ]:
alt.Chart(shot_training_set.groupby("play pattern").mean().reset_index()).mark_bar().encode(
        alt.X('play pattern:N', axis = alt.Axis(title = "player pattern", labelAngle = 0)),
        alt.Y('num opponents within 5 yards:Q', axis = alt.Axis(title = 'mean num opponents within 5 yards'))
).properties(width = 850, height = 300, 
             title = "mean num opponents whithin 5 yards by play pattern"
).configure_axis(labelFontSize = 13, titleFontSize = 15
).configure_title(fontSize = 16)

Ok, so actually, shots resulting from counters, on average, don't have less players within 5 yards of the shot location than other play patterns. What about number of opponents between the shot location and the goal?

In [ ]:
alt.Chart(shot_training_set.groupby("play pattern").mean().reset_index()).mark_bar().encode(
        alt.X('play pattern:N', axis = alt.Axis(title = "play pattern", labelAngle = 0)),
        alt.Y('num opponents between shot and goal:Q', axis = alt.Axis(title = 'num opponents between shot and goal'))
).properties(width = 850, height = 300, 
             title = "mean num opponents between shot and goal by play pattern"
).configure_axis(labelFontSize = 13, titleFontSize = 15
).configure_title(fontSize = 16)

Right, so we can see that actually, shots resulting from counters result in a higher rate of goals scored because there are less opponents between the shot location and the goal (on average), makes sense. 

#### 2.3.3 Percent goals by first time 

In [ ]:
plot_percent_goal_vs_attr("first time", "bar")

Above, we see that shots taken "first time" have a higher chance of resulting in a goal that those not taken first time. If all things were equal, this wouldn't make much intuitive sense. But in this case, shots taken first time likely had other favorable conditions which would increase their chance of resulting in a goal. Let's see if this is true:

In [ ]:
shot_training_set.groupby("first time").mean().reset_index()

In [ ]:
alt.Chart(shot_training_set.groupby("first time").mean().reset_index()).mark_bar().encode(
        alt.X('first time:N', axis = alt.Axis(title = "first time", labelAngle = 0)),
        alt.Y('Distance from goal:Q', axis = alt.Axis(title = 'Distance from goal'))
).properties(width = 850, height = 300, 
             title = "Mean distance from goal for shots taken first time vs not taken first time"
).configure_axis(labelFontSize = 13, titleFontSize = 15
).configure_title(fontSize = 16)

In [ ]:
alt.Chart(shot_training_set.groupby("first time").mean().reset_index()).mark_bar().encode(
        alt.X('first time:N', axis = alt.Axis(title = "player pattern", labelAngle = 0)),
        alt.Y('num opponents within 5 yards:Q', axis = alt.Axis(title = 'mean num opponents within 5 yards'))
).properties(width = 850, height = 300, 
             title = "Mean num opponents within 5 yards for shots taken first time vs not taken first time"
).configure_axis(labelFontSize = 13, titleFontSize = 15
).configure_title(fontSize = 16)

In [ ]:
alt.Chart(shot_training_set.groupby("first time").mean().reset_index()).mark_bar().encode(
        alt.X('first time:N', axis = alt.Axis(title = "player pattern", labelAngle = 0)),
        alt.Y('num opponents between shot and goal:Q', axis = alt.Axis(title = 'num opponents between shot and goal'))
).properties(width = 850, height = 300, 
             title = "Mean num opponents between shot and goal for shots taken first time vs not taken first time"
).configure_axis(labelFontSize = 13, titleFontSize = 15
).configure_title(fontSize = 16)

So just as we guessed, shots taken first time are, on average, slightly closer to the goal and are taken with less opponents between the shot location and the goal, and within 5 yards of the shot location.

## 3. Data preprocessing

So in the previous section, we explored our data, and most of our findings made intuitive sense. Now, we will create pipelines for preprocessing the categorical and numerical attributes in our shot data frame, then combine the two pipelines. To refresh our memories, here is the shot data frame again:

In [ ]:
shot_training_set.head()

### 3.1 Preprocessing pipeline

First, we create a class `AttrSelector`, which has methods `fit` and `transform` that we can use in our pipeline. This method will be use to select the features in our data frame for each pipeline.

Also, by making this class a subclass of `BaseEstimator` and `TransformerMixin`, we get the `fit_transform`, `get_params()` and `set_params()` methods as well.

In [ ]:
class AttrSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, attributes):
        self.attributes = attributes
   
    def fit(self, attributes):
        return self
    
    def transform(self, X):
        return X[self.attributes]

Now we will create a function which wraps a categorical and numerical pipeline into one full pipeline.

In our categorical pipeline, we will use AttrSelector to select the relevant categorical attributes, stored as a list in the _categorical_shot_attributes_ variable. We then encode the values of each categorical variable as "One hot vectors". In our numerical pipeline, we will use AttrSelector to select the relevant numerical attributes, stored as a list in the numerical_shot_attributes variable.

In [ ]:
def full_preprocessing_pipeline(X_train, X_test, categorical_features, numeric_features, fit=True):
    """
    """
    
    categorical_attr_pipeline = Pipeline([
        ('selector', AttrSelector(categorical_shot_attributes)),
        ('one_hot_encoder',  OneHotEncoder())
    ])
    
    numerical_attr_pipeline = Pipeline([
        ('selector', AttrSelector(numerical_shot_attributes)),
       #('poly', PolynomialFeatures(2)), 
        ('std_scaler', StandardScaler())
    ])
    
    combined_pipeline = ColumnTransformer(transformers=[('num', numerical_attr_pipeline, numeric_features),
                                                        ('cat', categorical_attr_pipeline, categorical_features)])
    
    if fit:
        X_train_preprocessed = combined_pipeline.fit_transform(X_train)
        X_test_preprocessed = combined_pipeline.transform(X_test)
    
        return X_train_preprocessed, X_test_preprocessed, combined_pipeline
    
    return combined_pipeline

In [ ]:
categorical_shot_attributes = ["play pattern", "technique", "first time"]
numerical_shot_attributes = ["x start location", "y start location", "x gk position", "y gk position", 
                             "num opponents within 5 yards", "num opponents between shot and goal", "Distance from goal"]

X_train_preprocessed, X_valid_preprocessed, preprocessor = full_preprocessing_pipeline(X_train=shot_training_set,
                                                                                       X_test=shot_valid_set,
                                                                                       categorical_features=categorical_shot_attributes,
                                                                                       numeric_features=numerical_shot_attributes,
                                                                                       fit=True)

In [ ]:
X_train_preprocessed

### 3.2 Get transformed features

In [ ]:
preprocessor
categorical_features_transformed = preprocessor.transformers[1][1]['one_hot_encoder'].fit(X_train[categorical_shot_attributes], y_train) \
                                                                                     .get_feature_names(categorical_shot_attributes)

#numerical_features_transformed = preprocessor.transformers[0][1]['poly'].fit(X_train[numerical_shot_attributes], y_train) \
#                                                                        .get_feature_names(numerical_shot_attributes)

all_features_transformed = list(numerical_shot_attributes) + list(categorical_features_onehot)

## 4. Models



### 4.1 Hyperparameter optimization 

In [ ]:
def hyperparameter_opt(classifier, hyperparameter_dist, X_train, y_train, scoring="f1", n_iter=10):
    """
    Runs randomized hyperparameter search on preprocessed data for specified classifer.
        
    Arguments:
    ----------
    classifier (sklearn classifier object)
        - classifer we want to optimize
    hyperparameter_dist (dict or list of dicts)
        - dictionary with parameters names (string) as keys and distributions or lists of hyperparameters to try
    X_train (pd.DataFrame)
        - training features from UCI's default of credit card clients Data Set.
        See here: http://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients# 
    y_train (pd.Series)
        - training labels from UCI's default of credit card clients Data Set
            
    Returns:
    --------
    (tuple) pd.DataFrame with hyperparameter cross validation results, and dict with best hyperparameters
    """ 

    print("Optimizing hyperparameters for {} model...".format(type(classifier).__name__))
    random_search = RandomizedSearchCV(estimator=classifier, param_distributions=hyperparameter_dist, cv=10, verbose=0, scoring=scoring, n_iter=n_iter)
    random_search.fit(X_train_preprocessed, y_train)
    print("Best hyperparameters {0}".format(random_search.best_params_))
    print("Best cross validation score {0}\n".format(random_search.best_score_))

    return pd.DataFrame(random_search.cv_results_), random_search.best_estimator_

Now let's optimize some classifers. Below, I am optimizing the hyperparameters using the 'f1-score' metric, since I want a balance of good precision and recall. 

In [ ]:
classifiers = {
    'lightgbm'      : (LGBMClassifier(),                    [{"max_depth" :  np.arange(1, 15),
                                                              "num_leaves" :  np.arange(0, 200),
                                                              "learning_rate" : uniform(0, 0.5)}]),
    'xgboost'       : (XGBClassifier(),                     [{"max_depth" :  np.arange(1, 15),
                                                              "num_leaves" :  np.arange(0, 200),
                                                              "eta" : uniform(0, 1)}]),
    'logistic reg'  : (LogisticRegression(max_iter=10000),  [{"C" : uniform(0, 20000)}]),
    'random forest' : (RandomForestClassifier(n_jobs=2),    [{"n_estimators" : np.arange(1, 200)}])
}

#store results of hyperparameter search for each model
cv_results =  {'lightgbm' : None,
               'xgboost' : None,
               'logistic reg' : None,
               'random forest' : None}

for classifier_name, classifier_obj in classifiers.items(): 
    results = hyperparameter_opt(classifier=classifier_obj[0], 
                                 hyperparameter_dist=classifier_obj[1], 
                                 X_train=X_train_preprocessed, 
                                 y_train=y_train,
                                 scoring="neg_log_loss",
                                 n_iter=20)
    cv_results[classifier_name] = results

### 4.2 Optimized models validation set performance

In [ ]:
validation_set_predictions = {'lightgbm' : None,
                              'xgboost' : None,
                              'logistic reg' : None,
                              'random forest' : None,}

for classifier_name, classifier_obj in cv_results.items():
    classifier = cv_results[classifier_name][1]
    y_train_pred = classifier.predict_proba(X_train_preprocessed)
    y_valid_pred = classifier.predict_proba(X_valid_preprocessed)

    validation_set_predictions[classifier_name] = y_valid_pred
    
    print(classifier_name)
    print("training set results: {0}".format(log_loss(y_train, y_train_pred, normalize=True)))
    print("validation set results: {0}".format(log_loss(y_valid, y_valid_pred, normalize=True)))
    print("")

In [ ]:
y_valid_pred = validation_set_predictions["xgboost"]
shot_valid_set_results = shot_valid_set.copy()
shot_valid_set_results["my xg"] = y_valid_pred[:, 1]

fig, ax = plt.subplots(1, 1, figsize=(10,10))
shot_valid_set_results.plot("statsbomb xg", "my xg", kind="scatter", ax=ax)
ax.plot([0, 1], [0, 1], color='red')

In [ ]:
shot_valid_set_results.sort_values("my xg", ascending=False).iloc[0:6]

In [ ]:
ascending=False
season_ids = shot_valid_set_results.sort_values("my xg", ascending=ascending).reset_index().iloc[1:7]["season_id"]
game_ids = shot_valid_set_results.sort_values("my xg", ascending=ascending).reset_index().iloc[1:7]["game_id"]
shot_ids = shot_valid_set_results.sort_values("my xg", ascending=ascending).reset_index().iloc[1:7]["shot id"]
xg_vals = shot_valid_set_results.sort_values("my xg", ascending=ascending).reset_index().iloc[1:7]["my xg"]

num=1
fig, ax = plt.subplots(2, 3, figsize=(36/1.2, 16/1.2))
for i in range(2):
    for j in range(3):
        ax_i = ax[i, j]
        draw_pitch(axis=ax_i, rotate=True)
        plot_shot_freeze_frame(la_liga[season_ids[num]][game_ids[num]].json_file, shot_ids[num], ax=ax_i)
        ax_i.annotate("Season: {}".format(season_ids[num]), xy = (1,75), size=16)
        ax_i.annotate("Game ID: {}".format(game_ids[num]), xy = (1,70), size=16)
        ax_i.annotate("xG: {}".format(round(xg_vals[num], 5)), xy = (1,65), size=16)
        num+=1
        
fig, ax = plt.subplots(1, 1, figsize=(12,8))
draw_pitch(axis=ax, rotate=True)
plot_shot_freeze_frame(la_liga["2018/2019"]['16215'].json_file, "6c30cac4-6f3b-4cd0-9913-26137b058416", ax=ax)

In [ ]:
ascending=True
season_ids = shot_valid_set_results.sort_values("my xg", ascending=ascending).reset_index().iloc[1:7]["season_id"]
game_ids = shot_valid_set_results.sort_values("my xg", ascending=ascending).reset_index().iloc[1:7]["game_id"]
shot_ids = shot_valid_set_results.sort_values("my xg", ascending=ascending).reset_index().iloc[1:7]["shot id"]
xg_vals = shot_valid_set_results.sort_values("my xg", ascending=ascending).reset_index().iloc[1:7]["my xg"]

num=1
fig, ax = plt.subplots(2, 3, figsize=(36/1.2, 16/1.2))
for i in range(2):
    for j in range(3):
        ax_i = ax[i, j]
        draw_pitch(axis=ax_i, rotate=True)
        plot_shot_freeze_frame(la_liga[season_ids[num]][game_ids[num]].json_file, shot_ids[num], ax=ax_i)
        ax_i.annotate("Season: {}".format(season_ids[num]), xy = (1,75), size=16)
        ax_i.annotate("Game ID: {}".format(game_ids[num]), xy = (1,70), size=16)
        ax_i.annotate("xG: {}".format(round(xg_vals[num], 5)), xy = (1,65), size=16)
        num+=1